<a href="https://colab.research.google.com/github/WilsonWKJ/Feature-Engineering/blob/main/Mastering_Feature_Engineering_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 職缺預測模型

## 載入套件

In [ ]:
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

--2024-01-15 07:05:28--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 172.217.12.14, 2607:f8b0:4025:815::200e
Connecting to drive.google.com (drive.google.com)|172.217.12.14|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2024-01-15 07:05:28--  https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.15.225, 2607:f8b0:4025:802::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.15.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659344 (20M) [application/octet-stream]
Saving to: ‘TaipeiSansTCBeta-Regular.ttf’

TaipeiSansTCBeta-Re 100%[===================>]  19.70M  61.7MB/s    in 0.3s    

2024-01-15 07:05:32 (61.7 MB

In [ ]:
import os
import re
import pandas as pd
import numpy as np


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm

import jieba

In [ ]:
fm.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

## 讀取資料

In [ ]:
training_set = pd.read_parquet('https://github.com/tlyu0419/DataScience/raw/master/04_Machine_Learning/data/Tutorial_training_set.parquet')
training_set = training_set.drop(['job_href', 'comp_href', 'activate_time'], axis=1)
print(training_set.shape)

testing_set = pd.read_parquet('https://github.com/tlyu0419/DataScience/raw/master/04_Machine_Learning/data/Tutorial_testing_set.parquet')
testing_set = testing_set.drop(['job_href', 'comp_href', 'activate_time'], axis=1)
print(testing_set.shape)

training_set.head(3)

(8067, 14)
(2017, 14)


,index,job_name,comp_name,job_desc,job_tags,profession,detail,work_years,fa_dollar_sign,location,fa_sitemap,fa_user,salary,mean_salary
3808,3808,Web 前端工程師,磐弈有限公司,【需求條件】 • 1 年以上工作相關經驗 • 熟悉 html / css / javascr...,[],軟體,front-end-engineer,1,4萬 ~ 7萬 TWD / 月,台中,不需負擔管理責任,中高階,"[40000.0, 70000.0]",55000.0
1478,1478,電聲產品-機構設計工程師(台北),美律實業股份有限公司,機構開發設計工作，主要開發產品：免持聽筒、耳機、藍牙耳機、可攜式音響系統、其他電聲產品 1....,"[excel, powerpoint, word]",工程研發,mechanical-engineer,3,4萬 ~ 7萬 TWD / 月,新北,不需負擔管理責任,中高階,"[40000.0, 70000.0]",55000.0
10705,10705,SA系統分析師,"賜鴻科技有限公司 Mores Information Technology Co., Ltd.",職務說明： 1. 負責客戶溝通與需求引導、歸納，並撰寫需求分析文件。 2. 負責專案系統之規...,"[full stack development, system analysis, prog...",軟體,system-architecture,2,6萬 ~ 8萬 TWD / 月,台北,不需負擔管理責任,中高階,"[60000.0, 80000.0]",70000.0


## 建模型

### EDA*

In [ ]:
plt.figure(figsize=(16, 10))

plt.subplot(2, 3, 1)
sns.scatterplot(data=training_set, x='mean_salary', y='work_years', alpha=0.03).set(xlim=(15000, 150000), ylim=(0, 10))

plt.subplot(2, 3, 2)
sns.boxplot(data=training_set, x='mean_salary', y='location').set(xlim=(15000, 125000))

plt.subplot(2, 3, 4)
sns.boxplot(data=training_set, x='mean_salary', y='fa_sitemap').set_xlim(15000, 250000)

plt.subplot(2, 3, 5)
sns.boxplot(data=training_set, x='mean_salary', y='fa_user').set_xlim(15000, 175000)

plt.subplot(2, 3, (3, 6))
sns.boxplot(data=training_set, x='mean_salary', y='profession').set_xlim(15000, 100000)

### Model Training - version 1

In [ ]:
formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user'
reg = sm.OLS.from_formula(formula, training_set).fit()
reg.summary2().tables[0]

In [ ]:
# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print('R^2 on Training Set', r2_score(training_set['mean_salary'], training_set['pred']))
print('MAE on Training Set', mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print('R^2 on Testing Set', r2_score(testing_set['mean_salary'], testing_set['pred']))
print('MAE on Testing Set', mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
sns.regplot(data=training_set, x='mean_salary', y='pred')

plt.subplot(1, 2, 2)
sns.regplot(data=testing_set, x='mean_salary', y='pred')

#### 錯誤分析1*

In [ ]:
training_set['res'] = training_set['pred'] - training_set['mean_salary']
training_set.sort_values('res')[['comp_name','job_name', 'job_desc', 'fa_user', 'fa_dollar_sign', 'mean_salary', 'pred', 'res']].round().head(20)

In [ ]:
training_set.sort_values('res')[['comp_name','job_name', 'job_desc', 'fa_user', 'fa_sitemap', 'fa_dollar_sign', 'mean_salary', 'pred', 'res']].round().tail(15)

### Model Training - version 2

#### 移除離群值

In [ ]:
training_set = training_set.loc[training_set['job_desc'].apply(lambda x: '職缺測試，請勿應徵。' not in x)]
testing_set = testing_set.loc[testing_set['job_desc'].apply(lambda x: '職缺測試，請勿應徵。' not in x)]

In [ ]:
# 丟掉最小薪資比最低工資還低的職缺
training_set = training_set.loc[training_set['salary'].apply(lambda x: min(x)>26400)]
testing_set = testing_set.loc[testing_set['salary'].apply(lambda x: min(x)>26400)]

In [ ]:
q1 = training_set['mean_salary'].quantile(0.25)
q3 = training_set['mean_salary'].quantile(0.75)
iqr = q3-q1
print(f'q1: {q1}, q3: {q3}, iqr: {iqr}')

In [ ]:
print(training_set.shape)
training_set = training_set.loc[training_set['mean_salary'].apply(lambda x: q1-1.5*iqr <= x <= q3+1.5*iqr)]
print(training_set.shape)

print(testing_set.shape)
testing_set = testing_set.loc[testing_set['mean_salary'].apply(lambda x: q1-1.5*iqr <= x <= q3+1.5*iqr)]
print(testing_set.shape)

In [ ]:
formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user'
reg = sm.OLS.from_formula(formula, training_set).fit()

In [ ]:
# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
sns.regplot(data=training_set, x='mean_salary', y='pred')

plt.subplot(1, 2, 2)
sns.regplot(data=testing_set, x='mean_salary', y='pred')

#### 特徵交互

In [ ]:
formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user + fa_user:profession'
reg = sm.OLS.from_formula(formula, training_set).fit()

In [ ]:
# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

In [ ]:
reg.summary2()

#### 錯誤分析v2*

In [ ]:
formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user'
reg = sm.OLS.from_formula(formula, training_set).fit()
training_set['pred'] = reg.predict(training_set)
testing_set['pred'] = reg.predict(testing_set)

In [ ]:
training_set['res'] = training_set['pred'] - training_set['mean_salary']
training_set.sort_values('res')[['job_name', 'job_desc', 'fa_dollar_sign', 'mean_salary', 'pred', 'res']].head(50)

In [ ]:
training_set.sort_values('res')[['job_name', 'job_desc', 'fa_dollar_sign', 'mean_salary', 'pred', 'res']].tail(50)

### Model Training - version 3

#### 移除業務性質的工作

In [ ]:
training_set['ratio'] = training_set['salary'].apply(lambda x: max(x)/min(x))
training_set.sort_values('ratio', ascending=False)[['profession', 'job_name', 'job_desc', 'fa_dollar_sign', 'mean_salary', 'ratio']].head(20)

In [ ]:
sum(training_set['ratio']>=3)

In [ ]:
print(training_set.shape)
training_set = training_set.loc[training_set['ratio']<3]
print(training_set.shape)

print(testing_set.shape)
testing_set['ratio'] = testing_set['salary'].apply(lambda x: max(x)/min(x))
testing_set = testing_set.loc[testing_set['ratio']<3]
print(testing_set.shape)

In [ ]:
formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user'
reg = sm.OLS.from_formula(formula, training_set).fit()

# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

In [ ]:
training_set = training_set.drop('ratio', axis=1)
testing_set = testing_set.drop('ratio', axis=1)

#### 用 Domain 從職缺描述中抽 Tag

In [ ]:
# 直接用 Domain 抽關鍵詞
training_set['python'] = training_set['job_desc'].apply(lambda x: 'python' in x)
training_set['javascript'] = training_set['job_desc'].apply(lambda x: 'javascript' in x)
training_set['linux'] = training_set['job_desc'].apply(lambda x: 'linux' in x)
training_set['sql'] = training_set['job_desc'].apply(lambda x: 'sql' in x)
training_set['word'] = training_set['job_desc'].apply(lambda x: 'word' in x)
training_set['excel'] = training_set['job_desc'].apply(lambda x: 'excel' in x)
training_set['powerpoint'] = training_set['job_desc'].apply(lambda x: 'powerpoint' in x)

formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user + python + javascript + linux + sql + word + excel + powerpoint'
reg = sm.OLS.from_formula(formula, training_set).fit()

# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

In [ ]:
testing_set['python'] = testing_set['job_desc'].apply(lambda x: 'python' in x)
testing_set['javascript'] = testing_set['job_desc'].apply(lambda x: 'javascript' in x)
testing_set['linux'] = testing_set['job_desc'].apply(lambda x: 'linux' in x)
testing_set['sql'] = testing_set['job_desc'].apply(lambda x: 'sql' in x)
testing_set['word'] = testing_set['job_desc'].apply(lambda x: 'word' in x)
testing_set['excel'] = testing_set['job_desc'].apply(lambda x: 'excel' in x)
testing_set['powerpoint'] = testing_set['job_desc'].apply(lambda x: 'powerpoint' in x)

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

In [ ]:
training_set = training_set.drop(['python', 'javascript', 'linux', 'sql', 'word', 'excel', 'powerpoint'], axis=1)
testing_set = testing_set.drop(['python', 'javascript', 'linux', 'sql', 'word', 'excel', 'powerpoint'], axis=1)

#### 用職缺 Tag 當標籤

In [ ]:
tags_cnt = training_set['job_tags'].explode().value_counts().reset_index()
tags_cnt = tags_cnt.loc[tags_cnt['job_tags']!='nan']
print(tags_cnt['index'][:60].values)
tags_cnt.head(20)

In [ ]:
kw_list = ['excel', 'word', 'powerpoint', 'outlook',
           'photoshop', 'illustrator', 'figma', 'premiere',
           'linux', 'android', 'ios', 'flutter',
           'python', 'javascript', 'java', 'c#', 'php', 'sql', 'c++', 'git', 'golang', 'vue.js', 'c', 'node.js',
           'google analytics', 'react', 'css', 'asp.net', 'html',
           'gcp', 'aws',
           'laravel', 'autocad', 'jquery', 'devops', 'kotlin', 'sketch']
for kw in kw_list:
    jieba.add_word(kw)

In [ ]:
training_set['job_desc_seg'] = training_set['job_desc'].apply(lambda x: jieba.lcut(x))
training_set['job_desc_seg'] = training_set.apply(lambda x: list(x['job_desc_seg']) + list(x['job_tags']), axis=1)
training_set['job_desc_seg'] = training_set['job_desc_seg'].apply(lambda x: list(set(x)))
training_set['job_desc_seg'] = training_set['job_desc_seg'].apply(lambda x: [i for i in x if i in kw_list])
training_set.head(3)

In [ ]:
testing_set['job_desc_seg'] = testing_set['job_desc'].apply(lambda x: jieba.lcut(x))
testing_set['job_desc_seg'] = testing_set.apply(lambda x: list(x['job_desc_seg']) + list(x['job_tags']), axis=1)
testing_set['job_desc_seg'] = testing_set['job_desc_seg'].apply(lambda x: list(set(x)))
testing_set['job_desc_seg'] = testing_set['job_desc_seg'].apply(lambda x: [i for i in x if i in kw_list])
testing_set.head(3)

In [ ]:
# Create one-hot column
for tag in kw_list:
    training_set[tag] = training_set['job_desc_seg'].apply(lambda x: tag in x)
    testing_set[tag] = testing_set['job_desc_seg'].apply(lambda x: tag in x)

rename_dict = {'c#':'c_sharp', 'c++':'c_plusplus', 'vue.js':'vue_js',
               'node.js':'node_js', 'google analytics':'google_analytics',
               'asp.net':'asp_net'}

training_set = training_set.rename(rename_dict, axis=1)
testing_set = testing_set.rename(rename_dict, axis=1)
training_set.columns

In [ ]:
training_set.head(1)

,index,job_name,comp_name,job_desc,job_tags,profession,detail,work_years,fa_dollar_sign,location,...,asp_net,html,gcp,aws,laravel,autocad,jquery,devops,kotlin,sketch
3808,3808,Web 前端工程師,磐弈有限公司,【需求條件】 • 1 年以上工作相關經驗 • 熟悉 html / css / javascr...,[],軟體,front-end-engineer,1,4萬 ~ 7萬 TWD / 月,台中,...,False,True,False,False,False,False,False,False,False,False


In [ ]:
' + '.join(training_set.columns[17:])

In [ ]:
formula = 'mean_salary ~ profession + work_years + location + fa_sitemap + fa_user' + ' + ' + ' + '.join(training_set.columns[17:])
reg = sm.OLS.from_formula(formula, training_set).fit()

In [ ]:
# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

#### 錯誤分析v3
- 其實可以再繼續做錯誤分析，但這邊先跳過

## 模型解析

In [ ]:
reg.summary2()

#### Create Dummies

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

features = ['profession', 'location', 'fa_sitemap', 'fa_user']
X = training_set[features]
enc.fit(X)

In [ ]:
enc.get_feature_names_out()

In [ ]:
training_dummies = pd.DataFrame(data=enc.transform(training_set[features]).toarray(),
                                columns=enc.get_feature_names_out())
training_dummies = training_dummies.drop(['profession_其他', 'location_其他縣市', 'fa_sitemap_不需負擔管理責任', 'fa_user_助理'], axis=1)


testing_dummies = pd.DataFrame(data=enc.transform(testing_set[features]).toarray(),
                                columns=enc.get_feature_names_out())
testing_dummies = testing_dummies.drop(['profession_其他', 'location_其他縣市', 'fa_sitemap_不需負擔管理責任', 'fa_user_助理'], axis=1)

training_dummies

In [ ]:
training_set = pd.concat([training_set.reset_index(drop=True), training_dummies], axis=1)
testing_set = pd.concat([testing_set.reset_index(drop=True), testing_dummies], axis=1)
training_set.columns

### 確認模型效度一致

In [ ]:
features = ['work_years', 'excel', 'word', 'powerpoint', 'outlook', 'photoshop', 'illustrator', 'figma',
            'premiere', 'linux', 'android', 'ios', 'flutter', 'python', 'javascript', 'java', 'c_sharp',
            'php', 'sql', 'c_plusplus', 'git', 'golang', 'vue_js', 'c', 'node_js', 'google_analytics',
            'react', 'css', 'asp_net', 'html', 'gcp', 'aws', 'laravel', 'autocad', 'jquery', 'devops',
            'kotlin', 'sketch', 'profession_人資', 'profession_客服', 'profession_工程研發', 'profession_建設',
            'profession_政府機關', 'profession_教育', 'profession_文字編輯', 'profession_法律', 'profession_物流貿易',
            'profession_生物醫療', 'profession_經營管理', 'profession_行銷', 'profession_製造', 'profession_設計',
            'profession_軟體', 'profession_遊戲製作', 'profession_金融', 'profession_餐飲',  'profession_業務', 'location_台中',
            'location_台北', 'location_台南', 'location_新北', 'location_新竹', 'location_桃園', 'location_高雄',
            'fa_sitemap_管理10_15人', 'fa_sitemap_管理15人以上', 'fa_sitemap_管理1_5人', 'fa_sitemap_管理5_10人',
            'fa_sitemap_管理人數未定', 'fa_user_中高階', 'fa_user_初階', 'fa_user_經營層', 'fa_user_經理']

formula = 'mean_salary ~ ' + ' + '.join(features)
# formula
reg = sm.OLS.from_formula(formula, training_set).fit()

In [ ]:
reg.summary2()

In [ ]:
reg_table = reg.summary2().tables[1]
reg_table

In [ ]:
# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

### Stepwise

In [ ]:
features = ['work_years', 'excel', 'word', 'powerpoint', 'outlook', 'photoshop', 'illustrator', 'figma',
            'premiere', 'linux', 'android', 'ios', 'flutter', 'python', 'javascript', 'java', 'c_sharp',
            'php', 'sql', 'c_plusplus', 'git', 'golang', 'vue_js', 'c', 'node_js', 'google_analytics',
            'react', 'css', 'asp_net', 'html', 'gcp', 'aws', 'laravel', 'autocad', 'jquery', 'devops',
            'kotlin', 'sketch', 'profession_人資', 'profession_客服', 'profession_工程研發', 'profession_建設',
            'profession_政府機關', 'profession_教育', 'profession_文字編輯', 'profession_法律', 'profession_物流貿易',
            'profession_生物醫療', 'profession_經營管理', 'profession_行銷', 'profession_製造', 'profession_設計',
            'profession_軟體', 'profession_遊戲製作', 'profession_金融', 'profession_餐飲', 'profession_業務',
            'location_台中', 'location_台北', 'location_台南', 'location_新北', 'location_新竹', 'location_桃園', 'location_高雄',
            'fa_sitemap_管理10_15人', 'fa_sitemap_管理15人以上', 'fa_sitemap_管理1_5人', 'fa_sitemap_管理5_10人',
            'fa_sitemap_管理人數未定', 'fa_user_中高階', 'fa_user_初階', 'fa_user_經營層', 'fa_user_經理']

In [ ]:
while True:
    formula = 'mean_salary ~ ' + ' + '.join(features)
    reg = sm.OLS.from_formula(formula, training_set).fit()
    reg_table = reg.summary2().tables[1]

    if reg_table['P>|t|'].max() > 0.05:
        c = reg_table['P>|t|'].idxmax().split('[')[0]
        features.remove(c)
        print(f'exclude: {c}')
    else:
        break

In [ ]:
reg.summary2()

In [ ]:
# 訓練集效度
training_set['pred'] = reg.predict(training_set)
print(r2_score(training_set['mean_salary'], training_set['pred']))
print(mean_absolute_error(training_set['mean_salary'], training_set['pred']))

# 測試集效度
testing_set['pred'] = reg.predict(testing_set)
print(r2_score(testing_set['mean_salary'], testing_set['pred']))
print(mean_absolute_error(testing_set['mean_salary'], testing_set['pred']))

## 其他優化模型效度的方式
1. 繼續做錯誤分析
2. 特徵
    - Target encoding
    - 葉編碼
    - stacking
3. 模型面
    - Tree-based model
    - NN